## Mosaick score maps

Test out code to mosaick score maps for version 2 maps

In [1]:
import rasterio
import numpy as np
from rasterio.io import MemoryFile
from rasterio.merge import merge
from rasterio.plot import show
import boto3
from rasterio.plot import show
import os
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
import re
from subprocess import run

In [5]:
def list_objects(s3_resource, bucket, prefix, suffix=None):
    """
    Get list of keys in an S3 bucket, filtering by prefix and suffix. Function
    developed by Kaixi Zhang as part of AWS_S3 class and adapted slightly here.
    This function retrieves all matching objects, and is not subject to the 1000
    item limit.
        Params:
            s3_resource (object): A boto3 s3 resource object
            bucket (str): Name of s3 bucket to list
            prefix (str): Prefix within bucket to search
            suffix (str, list): Optional string or string list of file endings
        Returns:
            List of s3 keys
    """
    keys = []
    if s3_resource is not None:
        s3_bucket = s3_resource.Bucket(bucket)
        for obj in s3_bucket.objects.filter(Prefix=prefix):
            # if no suffix given, add all objects with the prefix
            if suffix is None:
                keys.append(str(obj.key))
            else:
                # add all objects that ends with the given suffix
                if isinstance(suffix, list):
                    for _suffix in suffix:
                        if obj.key.endswith(_suffix):
                            keys.append(str(obj.key))
                            break
                else:
                    # suffix is a single string
                    if obj.key.endswith(suffix):
                        keys.append(str(obj.key))
    else:
        print
        'Warning: please first create an s3 resource'
    return keys

# Write mosaic to s3
def mosaic_to_s3(images, bucket, output_key, s3_client):

    # merge
    print('Merging images')
    array, out_trans = merge(images) 

    # profile 
    profile = images[0].profile
    profile['transform'] = out_trans
    profile['height'] = array.shape[1]
    profile['width'] = array.shape[2]
    profile['driver'] = 'GTiff'
    profile['compress'] = "lzw"
    profile['tiled'] = True
    
    # profile['dtype'] = 'int8'
    
    
    print('Writing to S3')
    # write to S3 (should be local, but don't have disk space)
    with MemoryFile() as mem_file:
        with mem_file.open(**profile) as dataset:
            dataset.write(array)

        # to s3 
        s3_client.upload_fileobj(mem_file, bucket, output_key)


# Mosaick local
def mosaic(images, fileout):

    # merge
    print('Merging images')
    array, out_trans = merge(images) 

    # profile 
    profile = images[0].profile
    profile['transform'] = out_trans
    profile['height'] = array.shape[1]
    profile['width'] = array.shape[2]
    profile['driver'] = 'GTiff'
    profile['compress'] = "lzw"
    # profile['tiled'] = True
    # profile['dtype'] = 'int8'
    
    print('Writing to disk')
    # write to local
    
    try:
        with rasterio.open(fileout, 'w', **profile) as dst:
            dst.write(array)
            
    except:
        print('Write failure for {}'.format(fileout))

### Get items in bucket

Set up paths and necessary credentials to read in

In [3]:
bucket = '***REMOVED***'
prefix_base = 'DL/Result/semantic_segmentation/boka/relabeller/coarse_as_benchmark/' + \
    'balanced_tversky_g09/new/d3500/refine_freeze58/ep150/predict/'
prefixes = ['{}predict_{}/Score_1/'.format(prefix_base, i) for i in range(1, 17)]

# set up output names and directories
# s3 location
output_prefix = 'maps/ghana/score_1/'
ROOT = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))

# local directory
local_dir = os.path.join(ROOT, 'imager/planet/catalog/')

# filenames
output_images = ['ghana_score1_{}_v0_1.tif'.format(i) for i in range(1, 17)]
# output_images
# local_dir


### Create mosaic

Going through just one prefix (AOI) as a test

#### List of input images

Get keys from S3 and set up output filenames

In [60]:
# client = boto3.client('s3')
s3 = boto3.resource('s3')
s3_client = boto3.client('s3') # client, for later use
keys = list_objects(s3, bucket, prefixes[0], 'tif')
keys_full = ['s3://***REMOVED***/%s' % (key) for key in keys]

# output file names
local_file = '{}{}'.format(local_dir, output_images[0])
cog_name = re.sub('.tif', '_cog.tif', output_images[0])
local_cog_file = '{}{}'.format(local_dir, cog_name)
output_key = '{}{}'.format(output_prefix, cog_name) # COG to S3

#### Read in images from s3

In [55]:
images = [rasterio.open(key) for key in keys_full]

#### Create mosaic

There is a way to do this straight onto S3 as well, but opting for local write (to EBS volume) to start

In [58]:
# mosaic_to_s3(images, bucket, output_key, s3_client)
mosaic(images, local_file)

Merging images
Writing to disk


#### Create COG

System call to `rio cogeo create`. Replace with python API and `cog_translate` in the long run, following approach [here](https://cogeotiff.github.io/rio-cogeo/API/). 

In [65]:
cmd = ['rio', 'cogeo', 'create', '--nodata', '-128', local_file, local_cog_file]
p = run(cmd, capture_output=True)
p.stderr.decode()

'Reading input: /home/ubuntu/projects/imager/planet/catalog/ghana_score1_1_v0_1.tif\nAdding overviews...\nUpdating dataset tags...\nWriting output to: /home/ubuntu/projects/imager/planet/catalog/ghana_score1_1_v0_1_cog.tif\n'

#### Validate COG

In [67]:
# subprocess.Popen(['rio', 'cogeo', 'validate', output_cog], stderr=subprocess.STDOUT)
cmd = ['rio', 'cogeo', 'validate', local_cog_file]
p = run(cmd, capture_output=True)
p.stdout.decode()
# p.stderr.decode()

'/home/ubuntu/projects/imager/planet/catalog/ghana_score1_1_v0_1_cog.tif is a valid cloud optimized GeoTIFF\n'

#### Upload to S3

In [68]:
s3_client.upload_file(local_cog_file, bucket, output_key)

## Process all

Once working, run in loop to process AOIs all at once

In [3]:
bucket = '***REMOVED***'
prefix_base = 'DL/Result/semantic_segmentation/boka/relabeller/coarse_as_benchmark/' + \
    'balanced_tversky_g09/new/d3500/refine_freeze58/ep150/predict/'

# local directory
ROOT = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))
local_dir = os.path.join(ROOT, 'imager/planet/catalog/')

# resource/client
s3 = boto3.resource('s3')
s3_client = boto3.client('s3') # client, for later use

#### Creating mosaics/COGs in loop

Run on a larger memory capacity instance. This almost got all the way through 16 AOIs using a t2.xlarge, but memory failure on one required 32 GB ram and and an r5.2xlarge. 

In [ ]:
# for i in range(1, 17): 
# for i in range(13, 14): 
for i in range(14, 17): 
    
#     i = 13
    # set up file paths
    input_prefix = '{}predict_{}/Score_1/'.format(prefix_base, i)
    output_prefix = 'maps/ghana/score_1/'  
    output_image = 'ghana_score1_{}_v0_1.tif'.format(i)
    
    # output files. Only COG goes to S3
    local_file = '{}{}'.format(local_dir, output_image)
    cog_name = re.sub('.tif', '_cog.tif', output_image)
    local_cog_file = '{}{}'.format(local_dir, cog_name)
    output_key = '{}{}'.format(output_prefix, cog_name) # COG to S3

    # image keys
    print("Getting keys {}".format(input_prefix))
    keys = list_objects(s3, bucket, input_prefix, 'tif')
    keys_full = ['s3://***REMOVED***/%s' % (key) for key in keys]
    
    # Read in images
    print("Reading images from {}".format(input_prefix))
    images = [rasterio.open(key) for key in keys_full]
    
    # create local mosaic
    print("Creating mosaic {}".format(local_file))
    mosaic(images, local_file)
    
    # create COG
    print("Creating cog from {}".format(local_file))
    cmd = ['rio', 'cogeo', 'create', '--nodata', '-128', local_file, local_cog_file]
    p = run(cmd, capture_output=True)
    print(p.stderr.decode())
    
    # validate COG
    print("Validating {}".format(local_cog_file))
    cmd = ['rio', 'cogeo', 'validate', local_cog_file]
    p = run(cmd, capture_output=True)
    print(p.stdout.decode())
    
    # upload to S3
    print("Uploading {} to S3".format(cog_name))
    s3_client.upload_file(local_cog_file, bucket, output_key)
    
    print("")

Getting keys DL/Result/semantic_segmentation/boka/relabeller/coarse_as_benchmark/balanced_tversky_g09/new/d3500/refine_freeze58/ep150/predict/predict_13/Score_1/
Reading images from DL/Result/semantic_segmentation/boka/relabeller/coarse_as_benchmark/balanced_tversky_g09/new/d3500/refine_freeze58/ep150/predict/predict_13/Score_1/
Creating mosaic /home/ubuntu/projects/imager/planet/catalog/ghana_score1_13_v0_1.tif
Merging images
Writing to disk
